# Statistiche di valutazione del modello addestrato

In [2]:
from medicane_utils.final_stats import evaluate_binary_classifier

In [3]:
import os
import torch
import numpy as np
import pandas as pd
from arguments import prepare_finetuning_args, Args
from dataset import build_dataset
from torch.utils.data import DataLoader
import models
from timm.models import create_model

from model_analysis import predict_label, get_path_pred_label, create_df_predictions, get_only_labels
from build_dataset import create_df_video_from_master_df, create_final_df_csv

output_dir = "../airmassRGB/supervised/" 

/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# devo ricreare un dataset perché serve un nuovo master df
df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])
df_data['delta_time'] = pd.to_timedelta(df_data['delta_time'])


/tmp/ipykernel_3581272/3092187031.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])


In [34]:
#df_data

In [12]:
df_videos = create_df_video_from_master_df(df_data, idxs=range(1,9), is_to_balance=True, output_dir=output_dir)

Num video CON cicloni: 85, Num video SENZA cicloni: 755	Totale video tiles: 840		Bilanciamento video...
 video senza cicloni tenuti: 85
Creazione delle folder per i 85 video...
Salvati 1120 file - Erano già presenti 240 file - File totali 1360
Creazione delle folder per i 85 video...
Salvati 0 file - Erano già presenti 1360 file - File totali 1360
170 video per il periodo da 2020-02-19 11:00:00 a 2020-02-19 12:15:00

Num video CON cicloni: 24, Num video SENZA cicloni: 444	Totale video tiles: 468		Bilanciamento video...
 video senza cicloni tenuti: 24
Creazione delle folder per i 24 video...
Salvati 352 file - Erano già presenti 32 file - File totali 384
Creazione delle folder per i 24 video...
Salvati 0 file - Erano già presenti 384 file - File totali 384
48 video per il periodo da 2020-03-12 20:00:00 a 2020-03-12 21:15:00

Num video CON cicloni: 21, Num video SENZA cicloni: 231	Totale video tiles: 252		Bilanciamento video...
 video senza cicloni tenuti: 21
Creazione delle folder per i

In [13]:
# Salvo il csv 
df_dataset_csv_test = create_final_df_csv(df_videos, output_dir)
df_dataset_csv_test.to_csv("./val_supervised_oggi3.csv", index=False)

In [8]:
# leggo il csv 
df_dataset_csv_test = pd.read_csv("./val_supervised_oggi3.csv") #, parse_dates=['datetime'])

In [14]:
# riprendo il df_video di tutti (senza ribilanciare) e poi faccio il merge
#df_videos = create_df_video_from_master_df(df_data, idxs=range(1,9), output_dir=None, is_to_balance=False)

In [15]:
df_videos

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths
420,420,639,0,19-02-2020_1215_639_0,1,2020-02-19 11:00:00,2020-02-19 12:15:00,"[../fromgcloud/airmass_rgb_20200219_1100.png, ..."
421,421,639,0,19-02-2020_1335_639_0,1,2020-02-19 12:20:00,2020-02-19 13:35:00,"[../fromgcloud/airmass_rgb_20200219_1220.png, ..."
422,422,639,0,19-02-2020_1455_639_0,1,2020-02-19 13:40:00,2020-02-19 14:55:00,"[../fromgcloud/airmass_rgb_20200219_1340.png, ..."
423,423,639,0,19-02-2020_1615_639_0,1,2020-02-19 15:00:00,2020-02-19 16:15:00,"[../fromgcloud/airmass_rgb_20200219_1500.png, ..."
424,424,639,0,19-02-2020_1735_639_0,1,2020-02-19 16:20:00,2020-02-19 17:35:00,"[../fromgcloud/airmass_rgb_20200219_1620.png, ..."
...,...,...,...,...,...,...,...,...
404,404,639,0,22-04-2020_1255_639_0,0,2020-04-22 11:40:00,2020-04-22 12:55:00,"[../fromgcloud/airmass_rgb_20200422_1140.png, ..."
627,627,1065,0,21-04-2020_1935_1065_0,0,2020-04-21 18:20:00,2020-04-21 19:35:00,"[../fromgcloud/airmass_rgb_20200421_1820.png, ..."
96,96,0,196,21-04-2020_1935_0_196,0,2020-04-21 18:20:00,2020-04-21 19:35:00,"[../fromgcloud/airmass_rgb_20200421_1820.png, ..."
241,241,426,0,20-04-2020_0055_426_0,0,2020-04-19 23:40:00,2020-04-20 00:55:00,"[../fromgcloud/airmass_rgb_20200419_2340.png, ..."


In [16]:
df_dataset_csv_test_p =df_dataset_csv_test.copy()
df_dataset_csv_test_p.path = df_dataset_csv_test.path.str.split('/').str.get(-1)

In [17]:

args = prepare_finetuning_args()
##### prende da val_supervised.csv
dataset_val, _ = build_dataset(is_train=False, test_mode=False, args=args)  

data_loader_val = DataLoader(
    dataset_val,
    batch_size=args.batch_size,
    shuffle=True,         # Per estrarre sample casuali
    num_workers=args.num_workers,
    pin_memory=args.pin_mem,
    drop_last=False
)

get_prediction = False

all_paths, all_preds, all_labels = get_only_labels(data_loader_val)
print(len(all_paths), len(all_preds), len(all_labels))

Number of the class = 2


890 890 890


In [18]:
# voglio prendere le predizioni
get_prediction = True
args.test_mode = True
args.init_ckpt = './output/checkpoint-best_850video.pth'
#args.init_ckpt = './output/checkpoint-best.pth'
# istanzia l'oggetto del modello 
print(f"Creating model: {args.model} (nb_classes={args.nb_classes})")
model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)

model.to(args.device)
model.eval()

Creating model: vit_giant_patch14_224 (nb_classes=2)
Caricamento del checkpoint da: ./output/checkpoint-best_850video.pth
Caricato state_dict con chiave: model
Checkpoint caricato con successo!


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1408, kernel_size=(2, 14, 14), stride=(2, 14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1408, out_features=4224, bias=False)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1408, out_features=1408, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1408, out_features=6144, bias=True)
        (act): GELU(approximate=none)
        (fc2): Linear(in_features=6144, out_features=1408, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
  

In [21]:
all_preds_last_model = all_preds

In [19]:
all_paths, all_preds, all_labels = get_path_pred_label(model, data_loader_val)

In [20]:
y_true = np.array(all_labels)
y_pred  = np.array(all_preds)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)
df_predictions.shape[0]

890

In [21]:
df_dataset_csv_test_p.merge(df_predictions, on='path')

,path,start,end,label,predictions,labels
0,19-02-2020_1215_639_0,1,16,1,0,1
1,19-02-2020_1335_639_0,1,16,1,0,1
2,19-02-2020_1455_639_0,1,16,1,0,1
3,19-02-2020_1615_639_0,1,16,1,1,1
4,19-02-2020_1735_639_0,1,16,1,1,1
...,...,...,...,...,...,...
885,22-04-2020_1255_639_0,1,16,0,0,0
886,21-04-2020_1935_1065_0,1,16,0,1,0
887,21-04-2020_1935_0_196,1,16,0,0,0
888,20-04-2020_0055_426_0,1,16,0,1,0


In [36]:
(y_true == y_pred).sum()/890

0.6730337078651686

# Modifica delle label dal Master df per verifica del periodo di formazione e fine del ciclone


In [24]:
from dataset_labeling_study import aggiorna_label_distanza_temporale
from build_dataset import get_gruppi_date, create_tile_videos, group_df_by_offsets
from dataset_labeling_study import create_tile_videos_relabeling, create_df_video_from_master_df_relabeling

In [8]:
df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])
#df_data.info()
df_data['delta_time'] = pd.to_timedelta(df_data['delta_time'])
df_data.shape[0]

/tmp/ipykernel_3570705/748525876.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_data = pd.read_csv("master_data_2020_wID.csv", parse_dates=['datetime', 'start_time', 'end_time', 'delta_time'])


279372

In [25]:
step = pd.Timedelta(minutes=40)
for i in range(1, 10):
    t = i * step
    print(t)
    aggiorna_label_distanza_temporale(df_data, soglia=t)

0 days 00:40:00
Colonna label_00_40 aggiunta
0 days 01:20:00
Colonna label_01_20 aggiunta
0 days 02:00:00
Colonna label_02_00 aggiunta
0 days 02:40:00
Colonna label_02_40 aggiunta
0 days 03:20:00
Colonna label_03_20 aggiunta
0 days 04:00:00
Colonna label_04_00 aggiunta
0 days 04:40:00
Colonna label_04_40 aggiunta
0 days 05:20:00
Colonna label_05_20 aggiunta
0 days 06:00:00
Colonna label_06_00 aggiunta


In [ ]:
#tile_mask = (df_data[['tile_offset_x','tile_offset_y']] == (852, 196)).all(axis=1)

In [26]:
#[['label', 'datetime', 'delta_time', 'label_00_20', 'label_03_00']]
# Seleziona tutte le colonne il cui nome inizia con 'label'
label_cols = [col for col in df_data.columns if col.startswith('label')]

# Crea un nuovo DataFrame solo con quelle colonne
#df_label = df_data.loc[tile_mask, label_cols+ ['path', 'tile_offset_x','tile_offset_y', 'datetime']]
df_label = df_data[label_cols+ ['path', 'tile_offset_x','tile_offset_y', 'datetime']]
df_label


,label,label_00_40,label_01_20,label_02_00,label_02_40,label_03_20,label_04_00,label_04_40,label_05_20,label_06_00,path,tile_offset_x,tile_offset_y,datetime
0,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,0,0,2020-01-01 12:00:00
1,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,213,0,2020-01-01 12:00:00
2,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,426,0,2020-01-01 12:00:00
3,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,639,0,2020-01-01 12:00:00
4,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20200101_1200.png,852,0,2020-01-01 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279367,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,213,196,2020-12-28 01:00:00
279368,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,426,196,2020-12-28 01:00:00
279369,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,639,196,2020-12-28 01:00:00
279370,0,0,0,0,0,0,0,0,0,0,../fromgcloud/airmass_rgb_20201228_0100.png,852,196,2020-12-28 01:00:00


In [27]:
df_videos_many_labels = create_df_video_from_master_df_relabeling(df_label)

Righe del df_video: 1044
Righe del df_video: 840
Righe del df_video: 468
Righe del df_video: 252
Righe del df_video: 468
Righe del df_video: 792
Righe del df_video: 708
Righe del df_video: 504
Righe del df_video: 708
Righe del df_video: 120
Righe del df_video: 1128
Righe del df_video: 48
Righe del df_video: 756
Righe del df_video: 864
Righe del df_video: 144
Righe del df_video: 60
Righe del df_video: 1464
Righe del df_video: 420
Righe del df_video: 612
Righe del df_video: 852
Righe del df_video: 1056
Righe del df_video: 1512
Righe del df_video: 648
Righe del df_video: 1080
Righe del df_video: 792


In [57]:
label_cols

['label',
 'label_00_40',
 'label_01_20',
 'label_02_00',
 'label_02_40',
 'label_03_20',
 'label_04_00',
 'label_04_40',
 'label_05_20',
 'label_06_00',
 'labels']

'path'

In [58]:
df_videos_many_labels[label_cols[:-1]+['path']]

,label,label_00_40,label_01_20,label_02_00,label_02_40,label_03_20,label_04_00,label_04_40,label_05_20,label_06_00,path
0,0,0,0,0,0,0,0,0,0,0,01-01-2020_1315_0_0
1,0,0,0,0,0,0,0,0,0,0,01-01-2020_1435_0_0
2,0,0,0,0,0,0,0,0,0,0,01-01-2020_1555_0_0
3,0,0,0,0,0,0,0,0,0,0,01-01-2020_1715_0_0
4,0,0,0,0,0,0,0,0,0,0,01-01-2020_1835_0_0
...,...,...,...,...,...,...,...,...,...,...,...
787,0,0,0,0,0,0,0,0,0,0,27-12-2020_1935_1065_196
788,0,0,0,0,0,0,0,0,0,0,27-12-2020_2055_1065_196
789,0,0,0,0,0,0,0,0,0,0,27-12-2020_2215_1065_196
790,0,0,0,0,0,0,0,0,0,0,27-12-2020_2335_1065_196


In [44]:
df_predictions

,path,predictions,labels
0,26-03-2020_0755_639_196,0,0
1,02-04-2020_1315_852_0,0,1
2,20-02-2020_1055_1065_0,0,0
3,25-03-2020_1315_0_196,0,0
4,31-03-2020_0115_1065_196,0,1
...,...,...,...
885,15-04-2020_0735_639_196,0,0
886,20-02-2020_1855_0_0,0,0
887,25-03-2020_1035_426_196,1,1
888,16-04-2020_0455_1065_0,1,0


In [59]:
df_result = df_videos_many_labels[label_cols[:-1]+['path']].merge(df_predictions, on='path')
#df_videos.merge(val_dataset, on='path')

In [ ]:
all(df_result.labels == df_result.labels), all(df_predictions.predictions == y_pred)

In [65]:
# allora posso rimuiovere la colonna labels
df_result.drop(columns='labels', inplace=True)

In [38]:
y_true = np.array(all_labels)
y_pred  = np.array(all_preds)
#np.save("y_true.npy",y_true)
#np.save("y_pred.npy",y_pred)


In [15]:
y_true = np.load("y_true.npy")
y_pred = np.load("y_pred.npy")

In [67]:
np.array(all_labels).shape # == y_true

(890,)

In [67]:
df_result[label_cols[:-1]]

,label,label_00_40,label_01_20,label_02_00,label_02_40,label_03_20,label_04_00,label_04_40,label_05_20,label_06_00
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
885,0,0,0,0,0,0,0,0,0,0
886,0,0,0,0,0,0,0,0,0,0
887,0,0,0,0,0,0,0,0,0,0
888,0,0,0,0,0,0,0,0,0,0


In [69]:
y_labels = df_result[label_cols[:-1]].values

In [72]:
y_labels.shape

(890, 10)

# Calcola le metriche

### metriche singole

In [23]:
results = evaluate_binary_classifier(y_true, y_pred) #[:,0])
print("\n=== Metriche ===")
print(results)

=== Confusion Matrix ===
[[332 113]
 [178 267]]
True Positive: 267
True Negative: 332
False Positive: 113
False Negative: 178

=== Classification Report ===
              precision    recall  f1-score   support

         neg       0.65      0.75      0.70       445
         pos       0.70      0.60      0.65       445

    accuracy                           0.67       890
   macro avg       0.68      0.67      0.67       890
weighted avg       0.68      0.67      0.67       890


=== Metriche ===
accuracy               0.673034
balanced_accuracy      0.673034
precision              0.702632
recall                 0.600000
f1                     0.647273
matthews_corrcoef      0.349819
cohen_kappa            0.346067
true_negatives       332.000000
false_positives      113.000000
false_negatives      178.000000
true_positives       267.000000
Name: metrics, dtype: float64


In [39]:
results = evaluate_binary_classifier(y_true, y_pred)
print("\n=== Metriche ===")
print(results)

=== Confusion Matrix ===
[[238 113]
 [ 47 304]]
True Positive: 304
True Negative: 238
False Positive: 113
False Negative: 47

=== Classification Report ===
              precision    recall  f1-score   support

         neg       0.84      0.68      0.75       351
         pos       0.73      0.87      0.79       351

    accuracy                           0.77       702
   macro avg       0.78      0.77      0.77       702
weighted avg       0.78      0.77      0.77       702


=== Metriche ===
accuracy               0.772080
balanced_accuracy      0.772080
precision              0.729017
recall                 0.866097
f1                     0.791667
matthews_corrcoef      0.554042
cohen_kappa            0.544160
true_negatives       238.000000
false_positives      113.000000
false_negatives       47.000000
true_positives       304.000000
Name: metrics, dtype: float64


### metriche al variare della label

In [ ]:
results = evaluate_binary_classifier(y_true, y_pred)
results.

=== Confusion Matrix ===
[[332 113]
 [178 267]]
True Positive: 267
True Negative: 332
False Positive: 113
False Negative: 178

=== Classification Report ===
              precision    recall  f1-score   support

         neg       0.65      0.75      0.70       445
         pos       0.70      0.60      0.65       445

    accuracy                           0.67       890
   macro avg       0.68      0.67      0.67       890
weighted avg       0.68      0.67      0.67       890



accuracy               0.673034
balanced_accuracy      0.673034
precision              0.702632
recall                 0.600000
f1                     0.647273
matthews_corrcoef      0.349819
cohen_kappa            0.346067
true_negatives       332.000000
false_positives      113.000000
false_negatives      178.000000
true_positives       267.000000
Name: metrics, dtype: float64

In [ ]:
for ylab in y_labels:
    
